In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
import json

from bokeh.models import GeoJSONDataSource,LinearColorMapper,NumeralTickFormatter
from bokeh.models import HoverTool,Select,ColorBar
from bokeh.palettes import brewer
from bokeh.io.doc import curdoc
from bokeh.layouts import widgetbox, row, column
from bokeh.plotting import figure

**Taxi zones data file being and read and Taxi zones in Manhattan being filtered**

In [2]:
tz_lookup = pd.read_csv('tz_lookup.csv')
tz_lookup = tz_lookup[tz_lookup['Borough'] == 'Manhattan']
tz_man = tz_lookup.LocationID.to_list()
## Only one origin is selected due to computation purposes. Can be scaled easily for more computation by varying the slicing index
tz_pu = tz_man[:1]

**Data being scraped from the Socrata API from NYC open data. The data limit has been set to 50000 for computation purpose. Can be extended accordingly**

In [3]:
template_str = "https://data.cityofnewyork.us/resource/2upf-qytp.json?$query=SELECT tpep_pickup_datetime as pu_time,tpep_dropoff_datetime as do_time, pulocationid as OTAZ,dolocationid as DTAZ, fare_amount as taxi_costs WHERE pulocationid in({}) AND dolocationid in ({}) limit 50000"
url = template_str.format(','.join(map(str,tz_pu)),','.join(map(str,tz_man)))

In [4]:
print(url)

https://data.cityofnewyork.us/resource/2upf-qytp.json?$query=SELECT tpep_pickup_datetime as pu_time,tpep_dropoff_datetime as do_time, pulocationid as OTAZ,dolocationid as DTAZ, fare_amount as taxi_costs WHERE pulocationid in(4) AND dolocationid in (4,12,13,24,41,42,43,45,48,50,68,74,75,79,87,88,90,100,103,104,105,107,113,114,116,120,125,127,128,137,140,141,142,143,144,148,151,152,153,158,161,162,163,164,166,170,186,194,202,209,211,224,229,230,231,232,233,234,236,237,238,239,243,244,246,249,261,262,263) limit 50000


In [5]:
resp = requests.get(url).json()

In [6]:
taxi_info_2019 = pd.DataFrame(resp)
taxi_info_2019.head()

,pu_time,do_time,OTAZ,DTAZ,taxi_costs
0,2019-12-31T20:11:28.000,2019-12-31T20:35:47.000,4,116,31.31
1,2019-12-31T19:57:00.000,2019-12-31T20:15:00.000,4,211,17.12
2,2019-12-30T19:27:36.000,2019-12-30T19:45:54.000,4,74,31.81
3,2019-12-30T12:40:15.000,2019-12-30T12:59:12.000,4,74,39.95
4,2019-12-31T07:28:00.000,2019-12-31T07:51:00.000,4,74,37.73


In [7]:
### Function to calculate taxi trip time given pickup and drop off time
def calc_taxi_time(row):
    interval = datetime.strptime(row['do_time'], "%Y-%m-%dT%H:%M:%S.000") - datetime.strptime(row['pu_time'], "%Y-%m-%dT%H:%M:%S.000")
    return int(interval.total_seconds())

In [8]:
##calculate taxi trip time for each data
taxi_info_2019['taxi_costs'] = taxi_info_2019.apply(lambda row: float(row['taxi_costs']),axis=1)
taxi_info_2019['taxi_time_sec'] = taxi_info_2019.apply(lambda row: calc_taxi_time(row),axis=1)
taxi_info_2019.head()

,pu_time,do_time,OTAZ,DTAZ,taxi_costs,taxi_time_sec
0,2019-12-31T20:11:28.000,2019-12-31T20:35:47.000,4,116,31.31,1459
1,2019-12-31T19:57:00.000,2019-12-31T20:15:00.000,4,211,17.12,1080
2,2019-12-30T19:27:36.000,2019-12-30T19:45:54.000,4,74,31.81,1098
3,2019-12-30T12:40:15.000,2019-12-30T12:59:12.000,4,74,39.95,1137
4,2019-12-31T07:28:00.000,2019-12-31T07:51:00.000,4,74,37.73,1380


**calculate aggregate taxi and time between each origin and destination pair**

In [9]:
agg_trips = taxi_info_2019.groupby(["OTAZ","DTAZ"]).agg({"taxi_costs":'mean',"taxi_time_sec":'mean'}).reset_index(level='DTAZ')
agg_trips.head()

,DTAZ,taxi_costs,taxi_time_sec
OTAZ,,,
4,100,15.411958,1263.464583
4,107,7.284834,661.326965
4,113,8.160755,927.087560
4,114,7.196331,525.635950
4,116,30.218519,1846.805556


In [10]:
tz_info = {}
for row in agg_trips.iterrows():
    otz = row[0]
    row = row[1]
    dest_info = tz_info.get(otz,{})
    dest_info[row['DTAZ']] = [row['taxi_costs'],row['taxi_time_sec']]
    tz_info[otz] = dest_info

**function to calculate uncertainity based on average value according to the following formula** <br>
uncertainity = actual_value-average value

In [11]:
def calc_uncertainity(row,cost=True):
    if cost:
        return row['taxi_costs'] - tz_info[row['OTAZ']][row['DTAZ']][0]
    else:
        return row['taxi_time_sec'] - tz_info[row['OTAZ']][row['DTAZ']][1]

In [12]:
## calculate uncertainity in taxi cost and time and average uncertainity for each origin and destination pair is being calculated
taxi_info_2019['cost_uncertainity'] = taxi_info_2019.apply(lambda row: calc_uncertainity(row,True),axis=1)
taxi_info_2019['time_uncertainity'] = taxi_info_2019.apply(lambda row: calc_uncertainity(row,False),axis=1)
agg_trips = taxi_info_2019.groupby(["OTAZ","DTAZ"]).agg({"taxi_costs":'mean',"taxi_time_sec":'mean','cost_uncertainity':'mean','time_uncertainity':'mean'}).reset_index()
agg_trips.head()

,OTAZ,DTAZ,taxi_costs,taxi_time_sec,cost_uncertainity,time_uncertainity
0,4,100,15.411958,1263.464583,2.102022e-15,-5.096960e-13
1,4,107,7.284834,661.326965,3.656772e-15,-1.821694e-12
2,4,113,8.160755,927.087560,-1.008318e-15,-3.033709e-13
3,4,114,7.196331,525.635950,-6.430118e-16,1.144667e-12
4,4,116,30.218519,1846.805556,2.111891e-14,4.210624e-14


#### attach geometry by ingesting taxi zones shape file to each origin and destination pair agg data 

In [13]:
taz = gpd.read_file('taxi_zones.shp')
taz = taz[taz['LocationID'].isin(tz_man)]

In [14]:
zone_shp = {}
for row in taz.iterrows():
    row = row[1]
    zone_shp[row['LocationID']] = row['geometry'] 

#### final data frame with average cost,time and uncertainity in time and cost

In [15]:
agg_trips['geometry'] = agg_trips.apply(lambda row: zone_shp[int(row['DTAZ'])],axis =1 )
agg_trips['OTAZ'] = agg_trips.apply(lambda row: int(row['OTAZ']),axis = 1 )
agg_trips['DTAZ'] = agg_trips.apply(lambda row: int(row['DTAZ']),axis = 1 )
agg_trips.head()

,OTAZ,DTAZ,taxi_costs,taxi_time_sec,cost_uncertainity,time_uncertainity,geometry
0,4,100,15.411958,1263.464583,2.102022e-15,-5.096960e-13,"POLYGON ((987770.527455166 212686.6776765436, ..."
1,4,107,7.284834,661.326965,3.656772e-15,-1.821694e-12,"POLYGON ((989131.6434821784 205749.9040614963,..."
2,4,113,8.160755,927.087560,-1.008318e-15,-3.033709e-13,"POLYGON ((986643.6403035522 204346.3236242086,..."
3,4,114,7.196331,525.635950,-6.430118e-16,1.144667e-12,"POLYGON ((986306.7117828578 203122.7862217873,..."
4,4,116,30.218519,1846.805556,2.111891e-14,4.210624e-14,"POLYGON ((1001062.708569705 241053.7687214315,..."


In [16]:
##converting final dataframe into geo data frame
agg_geoDf = gpd.GeoDataFrame(agg_trips)
agg_geoDf.set_geometry('geometry',inplace=True)
#agg_geoDf.crs = {'init': 'epsg:4326'}
agg_geoDf.head()

,OTAZ,DTAZ,taxi_costs,taxi_time_sec,cost_uncertainity,time_uncertainity,geometry
0,4,100,15.411958,1263.464583,2.102022e-15,-5.096960e-13,"POLYGON ((987770.527 212686.678, 987638.873 21..."
1,4,107,7.284834,661.326965,3.656772e-15,-1.821694e-12,"POLYGON ((989131.643 205749.904, 989084.531 20..."
2,4,113,8.160755,927.087560,-1.008318e-15,-3.033709e-13,"POLYGON ((986643.640 204346.324, 986592.535 20..."
3,4,114,7.196331,525.635950,-6.430118e-16,1.144667e-12,"POLYGON ((986306.712 203122.786, 986300.242 20..."
4,4,116,30.218519,1846.805556,2.111891e-14,4.210624e-14,"POLYGON ((1001062.709 241053.769, 1000940.914 ..."


### Visualization using Bokeh server

In [17]:
## function to get geoJson for visualization
def geoJsonDataSource(df,tz):
    df = df[df['OTAZ'] == tz]
    geoJson = json.loads(df.to_json())
    return json.dumps(geoJson)

In [18]:
##attribute map for each field
attr_map = {
    'cost': [0.0,50.0,'0.0','Avg cost and Uncertainity to each destination from {}'],
    'time': [0.0,1000.0,'0.0','Avg taxi time and Uncertainity in Seconds to each destination from {}']
}

In [19]:
### function to plot the map
def make_plot(field_name,tz,geosource,attr_map):
    attributes = attr_map[field_name]
    
    min_range = attributes[0]
    max_range = attributes[1]
    field_format = attributes[2]
    
    palette = brewer['Blues'][8]
    palette = palette[::-1]
    
    color_mapper = LinearColorMapper(palette = palette,low = min_range, high = max_range)
    
    format_tick = NumeralTickFormatter(format = field_format)
    color_bar = ColorBar(color_mapper = color_mapper,label_standoff = 18,formatter=format_tick,
                        border_line_color = None,location = (0,0))
    
    title = attributes[3].format(tz)
    
    plot = figure(title = title,
                 plot_height = 650,
                 plot_width = 850,
                 toolbar_location = None)
    plot.xgrid.grid_line_color = None
    plot.ygrid.grid_line_color = None
    plot.axis.visible = False
    color_fill = 'taxi_costs' if field_name == 'cost' else 'time_uncertainity'
    plot.patches('xs','ys',
                source = geosource,
                fill_color = {'field': color_fill,'transform':color_mapper},
                line_color = 'black',
                line_width = 0.25,
                fill_alpha = 1)
    plot.add_layout(color_bar,'right')
    
    hover = HoverTool(tooltips = [
    ('Dest Taxi Zone','@DTAZ'),
    ('Avg Taxi Cost','@taxi_costs'),
    ('Avg Time (secs)','@taxi_time_sec'),
    ('cost uncertainity','@cost_uncertainity'),
    ('time uncertainity','@time_uncertainity')
    ])
    plot.add_tools(hover)
    return plot
    

#### Plotting and adding the root layout to be displayed into Bokeh server

In [20]:
geosource = GeoJSONDataSource(geojson=geoJsonDataSource(agg_geoDf,tz_pu[0]))

palette = brewer['Blues'][8]
palette = palette[::-1]

hover = HoverTool(tooltips = [
    ('Dest Taxi Zone','@DTAZ'),
    ('Avg Taxi Cost','@taxi_costs'),
    ('Avg Time (secs)','@taxi_time_sec'),
    ('cost uncertainity','@cost_uncertainity'),
    ('time uncertainity','@time_uncertainity')
])

plot = make_plot('cost',str(tz_pu[0]),geosource,attr_map)

select_tz = Select(title = 'Select Taxi Zone: ', value = '100',options=list(map(str,tz_pu)))

select_asp = Select(title = 'Select Measure',value = 'cost',options = ['cost','time'])

layout = column(plot,widgetbox(select_tz))#,widgetbox(select_asp))
curdoc().add_root(layout)